In [18]:
# final version
import requests
import re
import json
import numpy
import time
import datetime

def lencheck(api_res):
    if api_res[-1]['createdAt'][0:10]>=until:
        return default_len
    else:        
        return numpy.argmax(list(map(lambda x:x['createdAt'][0:10]<until,api_res)))

    
    
def page_load(url,end = 30):
    res = requests.get(url,headers=header)
    return  json.loads(res.text)[0:end]


class Dcard_crawler(object):
    
    def __init__(self, forum, since, until, outfile, default_len=30, sleep=True):
        self.forumlist = ['timecapsule','relationship','girl','makeup','dressup','funny','rainbow','marvel','boy','horoscopes','food','mood','travel']
        self.forum = forum
        self.init_url = "https://www.dcard.tw/_api/forums/{}/posts?popular=false".format(forum) if forum in self.forumlist else None
        self.since = since if since >= '2015-01-01' else '2015-01-01'
        self.until = until if until <= str(datetime.datetime.now())[0:10] else str(datetime.datetime.now())[0:10]
        self.outfile = outfile
        self.default_len = default_len
        self.sleep = sleep
        self.post_ids = list()
        self.url_num = None
        self.data = dict()
        print("Dcard_crawler is activated...")
    
    def pid_collector(self):
        assert self.init_url,"the forum you type doesn't exist"
        page = requests.get(self.init_url,headers=header).json()
        self.post_ids.append([item['id'] for item in page])
        if lencheck(page) == self.default_len: # first page hasn't reach the end
            while True:
                iter_id = page[-1]['id']
                url = 'https://www.dcard.tw/_api/forums/{}/posts?popular=false&before={}'.format(self.forum,iter_id)
                page = page_load(url) if lencheck(page)==self.default_len else page_load(url,end=lencheck(page))
                self.post_ids.append([item['id'] for item in page])
                if self.sleep:
                    time.sleep(2)
                if len(page) != self.default_len:
                    break
        self.post_ids = sum(self.post_ids,[])
        self.url_num = len(self.post_ids)
        print(len(self.post_ids),"post urls are collected")
        
        
    def crawler(self):
        assert len(self.post_ids) > 0,'please run pid_collector first'
        self.data['info'] = {'until':self.until}
        self.data['post'] = [None]*self.url_num
        for idx,pid in enumerate(self.post_ids):
            # start of post crawler
            if p_idx % 20 == 0: print('crawling post {}...'.format(idx))
            p_url = 'https://www.dcard.tw/_api/posts/{}?'.format(pid)
            pos = requests.get(p_url,headers=header).json()
            time.sleep(2)
            # start of comment crawler
            total_pages = pos['commentCount']//30 + 1
            c_urls = ['https://www.dcard.tw/_api/posts/{}/comments?after={}'.format(pid,page*30) for page in range(total_pages)]
            c_res = [page_load(c_url) for c_url in c_urls]
            pos['comments'] = sum(c_res,[])
            self.data['post'][idx] = pos
        json.dump(self.data, open(self.outfile, 'w', encoding='utf-8'))
        print('process complete.')

In [19]:
header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}
forum = 'travel'
since = '2017-03-30'
default_len = 30

c = Dcard_crawler(forum,since)
c.pid_collector()
c.crawler()

Dcard_crawler is activated...
63 post urls are collected
crawling post 0...
crawling post 1...
crawling post 2...
crawling post 3...
crawling post 4...
crawling post 5...
crawling post 6...
crawling post 7...
crawling post 8...
crawling post 9...
crawling post 10...
crawling post 11...
crawling post 12...
crawling post 13...
crawling post 14...
crawling post 15...
crawling post 16...
crawling post 17...
crawling post 18...
crawling post 19...
crawling post 20...
crawling post 21...
crawling post 22...
crawling post 23...
crawling post 24...
crawling post 25...
crawling post 26...
crawling post 27...
crawling post 28...
crawling post 29...
crawling post 30...
crawling post 31...
crawling post 32...
crawling post 33...
crawling post 34...
crawling post 35...
crawling post 36...
crawling post 37...
crawling post 38...
crawling post 39...
crawling post 40...
crawling post 41...
crawling post 42...
crawling post 43...
crawling post 44...
crawling post 45...
crawling post 46...
crawling post

In [27]:
c.data['post'][5]['comments']

[{'anonymous': True,
  'content': '我剛去看了一下行程\n我都行動了哈哈哈',
  'createdAt': '2017-03-31T03:04:57.664Z',
  'floor': 1,
  'gender': 'F',
  'hidden': False,
  'host': False,
  'id': '84ad3fdd-bc09-4fe2-b375-d0b5055063f3',
  'inReview': False,
  'likeCount': 0,
  'postId': 226087311,
  'reportReason': '',
  'school': '中科大',
  'updatedAt': '2017-03-31T03:04:57.664Z',
  'withNickname': False},
 {'anonymous': True,
  'content': 'B1\n哈哈 你看了什麼行程阿',
  'createdAt': '2017-03-31T03:22:27.295Z',
  'floor': 2,
  'gender': 'M',
  'hidden': False,
  'host': True,
  'id': 'ae27f11c-7257-402b-aaa7-b4c14eb934d6',
  'inReview': False,
  'likeCount': 0,
  'postId': 226087311,
  'reportReason': '',
  'school': '逢甲大學',
  'updatedAt': '2017-03-31T03:22:27.295Z',
  'withNickname': False},
 {'anonymous': True,
  'content': 'b2\n自由行三日\n行程感覺還不錯\n\n坐船坐飛機都划算的感覺',
  'createdAt': '2017-03-31T03:34:23.444Z',
  'floor': 3,
  'gender': 'F',
  'hidden': False,
  'host': False,
  'id': 'a271f310-d931-4902-99cb-30049081fde8',
 

In [ ]:
import requests
import json
import numpy
import time

counter = 0
since = '2016-12-20'
default_len = 30
post_ids = []


def page_load(url,end=default_len):
    res = requests.get(url,headers=header)
    return  json.loads(res.text)[0:end]

def lencheck(api_res):
    if api_res[-1]['createdAt'][0:10]>=since:
        return default_len
    else:        
        return numpy.argmax(list(map(lambda x:x['createdAt'][0:10]<since,api_res)))

# def write_file(filename,encoding='utf-8'):
#     with open(filename,encoding=encoding,'r'):    

header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}
init_url = 'https://www.dcard.tw/f/travel'
res = requests.get(init_url,headers=header)
data = re.search('\"store\"\:(\[.+\"anonymousDepartment\"\:.+?\}\])',res.text).group(1)
print(json.loads(data)[1])
# page = page_load(init_url)  
# post_ids.append([item['id'] for item in page])

# if lencheck(page) == default_len: # first page hasn't reach the end
#     while True:
#         iter_id = page[-1]['id']
#         url = 'https://www.dcard.tw/_api/forums/travel/posts?popular=false&before={}'.format(iter_id)
#         page = page_load(url) if lencheck(page)==default_len else page_load(url,end=lencheck(page))
#         post_ids.append([item['id'] for item in page])
# #         time.sleep(3)
#         counter += len(page)
#         if len(page) != default_len:
#             print('complete')
#             break
# else:
#     print('complete')
    
# print("total count =",counter)
# post_ids = sum(post_ids,[])

In [ ]:
import re

for pid in post_ids[0:1]:
    p_url = 'https://www.dcard.tw/f/relationship/p/{}'.format(pid)
    p_res = requests.get(p_url,headers=header)
    data = json.loads(re.search('\"store\"\:\[(.+?\"anonymousDepartment\"\:.+?\})\]',p_res.text).group(1))
#     print(re.sub('\n',' ',data['content']))
    print(type(data['commentCount'])

In [ ]:
commentCount = 0
total_pages = commentCount//30 + 1
after = 0
for page in range(total_pages):
    c_url ='https://www.dcard.tw/_api/posts/225513354/comments?after={}'.format(page)
    c_res = page_load(c_url)
    print(c_res)
    after += 30

In [ ]:
def lencheck(api_res):
    if api_res[-1]['createdAt'][0:10]>=since:
        return default_len
    else:        
        return numpy.argmax(list(map(lambda x:x['createdAt'][0:10]<since,api_res)))

def page_load(url,end = 30):
    res = requests.get(url,headers=header)
    return  json.loads(res.text)[0:end]

In [ ]:
import requests
import re
import json
import numpy
import time
from collections import defaultdict

class Dcard_crawler(object):
    
    def __init__(self, init_url, since, default_len=30, sleep=True):
        self.init_url = init_url
        self.since = since
        self.default_len = default_len
        self.sleep = sleep
        self.post_ids = []
        self.url_num = None
        self.data = defaultdict(list)
        
    
#     def write_file(filename,res_type):
#         with open(filename,'w',encoding='utf-8'):
    
    
    def pid_collector(self):
        page = page_load(self.init_url)  
        self.post_ids.append([item['id'] for item in page])
        if lencheck(page) == self.default_len: # first page hasn't reach the end
            while True:
                iter_id = page[-1]['id']
                url = 'https://www.dcard.tw/_api/forums/travel/posts?popular=false&before={}'.format(iter_id)
                page = page_load(url) if lencheck(page)==self.default_len else page_load(url,end=lencheck(page))
                self.post_ids.append([item['id'] for item in page])
                if self.sleep :
                    time.sleep(2)
                if len(page) != self.default_len:
                    break
        self.post_ids = sum(self.post_ids,[])
        self.url_num = len(self.post_ids)
        print(len(self.post_ids),"urls are collected")
        
        
    def crawler(self):
        if len(self.post_ids) == 0:
            print('please run url_collector first')
        else:
            for idx,pid in enumerate(self.post_ids):
                # start of post crawler
                print('crawling post {}...'.format(idx))
                p_url = 'https://www.dcard.tw/f/relationship/p/{}'.format(pid)
                print(p_url)
                p_res = requests.get(p_url,headers=header)
                pos = json.loads(re.search('\"store\"\:\[(.+?\"anonymousDepartment\"\:.+?\})\]',p_res.text).group(1))
                print(pos)
                content = re.sub('\n',' ',pos['content'])
                out.write(content+'\n')
                if idx%2 == 0:
                    time.sleep(2)
                # start of comment crawler
                total_pages = pos['commentCount']//30 + 1
                after = 0
                with open("{}_comments.csv".format(pid),'w',encoding='utf-8') as com_f:
                    for page in range(total_pages):
                        c_url ='https://www.dcard.tw/_api/posts/{}/comments?after={}'.format(pid,page)
                        c_res = page_load(c_url)
                        for com in c_res:
                            if not com['hidden']:
                                com_f.write(re.sub('\n',' ',com['content']))                            
                        after += 30

In [7]:
import json
from pprint import pprint

with open('result.json','r',encoding='utf-8') as json_data:
    d = json.loads(json_data.read())

d['post'][5]

{'anonymousDepartment': False,
 'anonymousSchool': False,
 'commentCount': 9,
 'comments': [{'anonymous': True,
   'content': '看到 親愛的雞巴上ㄧ棒 瞬間大笑\n而且在兒童套餐啊！',
   'createdAt': '2017-01-05T23:05:13.872Z',
   'floor': 1,
   'gender': 'M',
   'hidden': False,
   'host': False,
   'id': '8a4a38b8-cc00-47c0-b896-f174bcf3312c',
   'inReview': False,
   'likeCount': 2,
   'postId': 225544910,
   'reportReason': '',
   'school': '東華大學',
   'updatedAt': '2017-01-05T23:06:17.768Z',
   'withNickname': False},
  {'anonymous': False,
   'content': '想看親愛的雞巴上一棒照片👋\U0001f3fb',
   'createdAt': '2017-01-05T23:14:17.322Z',
   'department': '政治學研究所',
   'floor': 2,
   'gender': 'F',
   'hidden': False,
   'host': False,
   'id': '94f90e35-5767-4de6-8376-d80b49bf178e',
   'inReview': False,
   'likeCount': 2,
   'postId': 225544910,
   'reportReason': '',
   'school': '國立臺灣師範大學',
   'updatedAt': '2017-01-05T23:36:39.993Z',
   'withNickname': False},
  {'anonymous': False,
   'content': '原來大家都很在意關鍵字',
   'crea